# Practical Machine Learning for Physicists
# Coursework D

In this notebook you will be trying to predict a system using incomplete information. We will set up the equations of motions for a simple double pendulum (or should that be a double simple pendulum. Then we will see if a machine learning technique can predict the future position of the lower mass, using only the lower mass positions.

### Kinematics of the double pendulum
Let's specify our problem in terms of the following, with the origin at the pivot point of the top pendulum. This is just background for the machine learning tasks at the bottom of the notebook.

#### Positions
$$x_1 = L_1 \sin \theta_1$$
$$y_1 = -L_1 \cos \theta_1$$
$$x_2 = x_1 + L_2 \sin \theta_2$$
$$y_2 = y_1 - L_2 \cos \theta_2$$

#### Velocities
$$\dot{x}_1 = \dot{\theta_1} L_1 \cos \theta_1$$
$$\dot{y_1} =  \dot{\theta_1} L_1 \sin \theta_1$$
$$\dot{x_2} = \dot{x_1} + \dot{\theta_2} L_2 \cos \theta_2$$
$$\dot{y_2} = \dot{y_1} + \dot{\theta_2} L_2 \sin \theta_2$$


#### Accelerations

$$\ddot{x}_1 = -\dot{\theta_1}^2 L_1 \sin \theta_1 + \ddot{\theta_1} L_1 \cos \theta_1$$
$$\ddot{y_1} =  \dot{\theta_1}^2 L_1 \cos \theta_1 + \ddot{\theta_1} L_1 \sin \theta_1$$
$$\ddot{x_2} = \ddot{x_1} - \dot{\theta_2}^2 L_2 \sin \theta_2 + \ddot{\theta_2} L_2 \cos \theta_2$$
$$\ddot{y_2} = \ddot{y_1} + \dot{\theta_2}^2 L_2 \cos \theta_2 + \ddot{\theta_2} L_2 \sin \theta_2$$

#### Energies
Let $v_1^2 = \dot{x_1}^2 +\dot{y_1}^2$ and $v_2^2 = \dot{x_2}^2 +\dot{y_2}^2$ then the kinetic energies $T_1$ and $T_2$ are
$$ T_1 = \frac{1}{2}m_1 v_1^2 = \frac{1}{2}m_1 L_1^2 \dot{\theta_1}^2 $$
$$ T_2 = \frac{1}{2}m_2 v_2^2 = \frac{1}{2}m_2 \left( L_1^2 \dot{\theta_1}^2 + L_2^2 \dot{\theta_2}^2 + 2L_1 L_2 \cos(\theta_1-\theta_2) \dot{\theta_1} \dot{\theta_2} \right) $$

The potential enrgies are
$$V_1 = m_1 g y_1 = - m_1 g L_1 \cos \theta_1$$
$$V_2 = m_2 g y_2 = -m_2 g ( L_1 \cos \theta_1 + L_2 \cos \theta_2)$$

#### Langrangian
Now we form the Lagrangian $L=T-V=T_1+T_2 -V_1 -V_2$ and use the Euler-Lagrange equations:
$$\frac{\partial L}{\partial \theta_1} = \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_1}}$$
$$\frac{\partial L}{\partial \theta_2} = \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_2}}$$

Applying these gives
$$-(m_1+m_2) g L_1 \sin \theta_1 = (m_1+m_2) L_1^2 \ddot{\theta_1} + m_2 L_1 L_2 \sin(\theta_1-\theta_2) \dot{\theta_2}^2 +  m_2 L_1 L_2 \cos(\theta_1-\theta_2) \ddot{\theta_2} $$
and
$$ -m_2 g L_2 \sin \theta_2 = m_2 L_2 \ddot{\theta_2} + m_2 L_1 L_2 \cos(\theta_1-\theta_2) \ddot{\theta_1} + m_2 L_1 L_2 \sin(\theta_1-\theta_2) \dot{\theta_1}^2 $$


#### Equations of motions
$$ \omega_1 = \dot{\theta_1}$$  

$$ \omega_2 = \dot{\theta_2}$$
$$ \ddot\theta_1 = \frac{1}{L_1\xi}\left[L_1m_2\cos(\theta_1-\theta_2)\sin(\theta_1-\theta_2)\omega_1^2 + L_2m_2\sin(\theta_1-\theta_2)\omega_2^2 - m_2g\cos(\theta_1-\theta_2)\sin(\theta_2) + (m_1+m_2)g\sin(\theta_1) \right] $$
$$ \ddot\theta_2 = \frac{1}{L_2\xi}\left[L_2m_2\cos(\theta_1-\theta_2)\sin(\theta_1-\theta_2)\omega_2^2 + L_1(m_1+m_2)\sin(\theta_1-\theta_2)\omega_1^2+(m_1+m_2)g\sin(\theta_1)\cos(\theta_1-\theta_2) - (m_1+m_2)g\sin(\theta_2) \right] $$
where
$$\xi \equiv \cos^2(\theta_1-\theta_2)m_2-m_1-m_2$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

import matplotlib.style #Some style nonsense
import matplotlib as mpl #Some more style nonsense

#Set default figure size
#mpl.rcParams['figure.figsize'] = [12.0, 8.0] #Inches... of course it is inches
mpl.rcParams["legend.frameon"] = False
mpl.rcParams['figure.dpi']=200 # dots per inch



In [ ]:
def rhs(t, z, L1, L2, m1, m2, g):
    """
    Returns the right-hand side of the ordinary differential equation describing the double pendulem
    """
    theta1, w1, theta2, w2 = z    #The four components ## w1 is theta_1 dot
    cos12 = np.cos(theta1 - theta2)
    sin12 = np.sin(theta1 - theta2)
    sin1 = np.sin(theta1)
    sin2 = np.sin(theta2)
    xi = cos12**2*m2 - m1 - m2 ## zeta
    w1dot = ( L1*m2*cos12*sin12*w1**2 + L2*m2*sin12*w2**2 ## theta_1 double dot
            - m2*g*cos12*sin2      + (m1 + m2)*g*sin1)/(L1*xi)
    w2dot = -( L2*m2*cos12*sin12*w2**2 + L1*(m1 + m2)*sin12*w1**2
            + (m1 + m2)*g*sin1*cos12  - (m1 + m2)*g*sin2 )/(L2*xi)
    return w1, w1dot, w2, w2dot   #Return the w's and the wdot's


def to_cartesian(theta1, w1, theta2, w2, L1, L2):
    """ Transforms theta and omega to cartesian coordinates
    and velocities x1, y1, x2, y2, vx1, vy1, vx2, vy2
    """
    x1 = L1 * np.sin(theta1)
    y1 = -L1 * np.cos(theta1)
    x2 = x1 + L2 * np.sin(theta2)
    y2 = y1 - L2 * np.cos(theta2)
    vx1 = L1*np.cos(theta1)*w1
    vy1 = L1*np.sin(theta1)*w1
    vx2 = vx1 + L2*np.cos(theta2)*w2
    vy2 = vy1 + L2*np.sin(theta2)*w2
    return x1, y1, x2, y2, vx1, vy1, vx2, vy2


In [ ]:
# Set up the initial conditions. Here we have lengths and masses
L1, L2 = 1., 1.
m1, m2 = 3., 1.
g = 9.81     # [m/s^2]. Gravitational acceleration

#Starting angles
z0=[np.pi/4,0,np.pi/4,0]
#z0=[0.1,0,0.1,0]

#Time ranges
tmax, dt = 50, 0.1
t = np.arange(0, tmax+dt, dt)

In [ ]:
# Solve initial value problem
ret = solve_ivp(rhs, (0,tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
z=ret.y
print(np.shape(z))

# Extract result
theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]
x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

In [ ]:
fig,ax=plt.subplots()
ax.plot(x1, y1, label=r"Track $m_1$")
ax.plot(x2, y2, label=r"Track $m_2$")
ax.plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", label="Initial position", c='k')
plt.ylabel(r"$y/L$")
plt.xlabel(r"$x/L$")
ax.legend()

# Exercises: Predicting Chaos
1. Design and train a recurrent neural network (of your choice) to predict the future positions, where future is defined as $t=t_0 + 20 \delta t$, of the masses $m_1$ and $m_2$ using their cartesian coordinates and the initial conditions  $z_0=[\pi/4,0,\pi/4,0]$.
2. How stable is your network to variations in initial conditions? Make a plot of $x$ and $y$ vs time to show the network prediction in comparison to the solution from solve_ivp
3. How far into the future can a network predict? Make a plot showing how the deviation between predicted position and actual position (from solve_ivp above) vary as a function of extrapolation time from $t=t_0 + 20 \delta t$ to $t=t_0 + 100 \delta t$  (e.g. for each extrapolation time, train a new version of the network and then plot the performance)
4. Repeat steps 1-3 for the initial conditions $z_0=[\pi/2,0,\pi/2,0]$ which give a much more complex path.
5. Repeat steps 1-4 but only train your neural network on the cartesian coordinates of the mass $m_2$ (i.e without showing your neural network the positions of the mass $m_1$)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

import matplotlib.style #Some style nonsense
import matplotlib as mpl #Some more style nonsense

#Import tqdm for progress bar
from tqdm import tqdm

#Set default figure size
#mpl.rcParams['figure.figsize'] = [12.0, 8.0] #Inches... of course it is inches
mpl.rcParams["legend.frameon"] = False
mpl.rcParams['figure.dpi']=200 # dots per inch

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Define the model
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(None, 8)))  # 8 input features per timestep for 2 masses
model.add(keras.layers.LSTM(50, return_sequences=True))  # Increased units and return sequences
model.add(keras.layers.Dropout(0.2)) # Added dropout for regularization
model.add(keras.layers.LSTM(50, return_sequences=False))  # Use only the last output
model.add(keras.layers.Dense(4, activation="linear"))  # 4 outputs
model.compile(loss='mean_squared_error', optimizer='adam') # you can try using different optimizers or learning rates

In [ ]:
# Plot Target Motion
numPoints=400
x1=x1[0:numPoints]
y1=y1[0:numPoints]
x2=x2[0:numPoints]
y2=y2[0:numPoints]
vx1=vx1[0:numPoints]
vy1=vy1[0:numPoints]
vx2=vx2[0:numPoints]
vy2=vy2[0:numPoints]

fig,ax=plt.subplots(figsize = (6,4))
ax.plot(x1,y1,label=r"$m_1$")
ax.plot(x2,y2,label=r"$m_1$")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Target Motion")
ax.legend()

In [ ]:
windowsize=20 #Number of samples we will use to train our network
offset=20 #How many samples into the future to predict

#This function splits up a 1-d array x into a series of overlapping windows
#The return is a tuple of the array of input windows and target (label) windows
def shapeArray(x,windowsize,offset):
    xInput= np.array([x[i : i + windowsize] for i in range(len(x)-(windowsize+offset)+1)])
    xLabel= np.array([x[i +windowsize : i+ windowsize+offset] for i in range(len(x)-(windowsize+offset)+1)])
    return (xInput,xLabel)

# Define a cut off for training, rest will be used for prediction later on
cutOff = 200

#Get our xInput and xLabel arrays
xInput1,xLabel1=shapeArray(x1[:cutOff],windowsize,offset)
xInput2,xLabel2=shapeArray(x2[:cutOff],windowsize,offset)
vx1Input,vx1Label=shapeArray(vx1[:cutOff],windowsize,offset)
vx2Input,vx2Label=shapeArray(vx2[:cutOff],windowsize,offset)


#Get our yInput and yLabel arrays
# We already have our outputs so we don't need to explicitly calculate them
yInput1,yLabel1=shapeArray(y1[:cutOff],windowsize,offset)
yInput2,yLabel2=shapeArray(y2[:cutOff],windowsize,offset)
vy1Input,vy1Label=shapeArray(vy1[:cutOff],windowsize,offset)
vy2Input,vy2Label=shapeArray(vy2[:cutOff],windowsize,offset)

#Print show the shape of the arrays
print(xInput1.shape) # 61 as we cut off at 80 and window size is 20
print(xLabel1.shape)
print(xInput1[0][:10])
print(xInput1[1][:10]) # shifted by one

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))
ax.plot(xInput1[0],yInput1[0],label=r"$m_1$")
ax.plot(xInput2[0],yInput2[0],label=r"$m_2$")
ax.plot(xLabel1[0],yLabel1[0],label=r"$m_1$ pred")
ax.plot(xLabel2[0],yLabel2[0],label=r"$m_2$ pred")

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Expected Prediction")
ax.legend()


In [ ]:
# Reshape Arrays so for training so that they can fit into our network
xInput1_Train=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
xInput2_Train=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
yInput1_Train=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
yInput2_Train=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
vx1Input_Train=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
vx2Input_Train=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
vy1Input_Train=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
vy2Input_Train=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

xLabel1_Train=xLabel1[:,-1].reshape(xLabel1.shape[0],1) # added reshape to (batch_size, 1, 1)
xLabel2_Train=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
yLabel1_Train=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
yLabel2_Train=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

# Deepest array contains values from all 8 inputs
y_in = np.concatenate([xInput1_Train, yInput1_Train, xInput2_Train, yInput2_Train, vx1Input_Train, vx2Input_Train, vy1Input_Train, vy2Input_Train], axis=2)
y_target = np.concatenate([xLabel1_Train, yLabel1_Train, xLabel2_Train, yLabel2_Train], axis=1)

In [ ]:
print(np.shape(y_in))
print(np.shape(y_target))

In [ ]:
steps=200  #Number of training steps
costs=np.zeros(steps)  #Array for plotting cost
for i in tqdm(range(steps)): # tqdm is progress bar
    costs[i]=model.train_on_batch(y_in,y_target) #Train the network

#Plot costs vs steps
fig,ax=plt.subplots(figsize=(6,4))
ax.plot(np.arange(steps),costs,label=r"Costs")
ax.set_xlabel("Steps")
ax.set_ylabel("Cost")
ax.set_title("Network Training Cost")
ax.legend()

In [ ]:
# Create test set, we only create about labels but we make the inputs anyway
xInput1,xLabel1=shapeArray(x1,windowsize,offset)
xInput2,xLabel2=shapeArray(x2,windowsize,offset)
vx1Input,vx1Label=shapeArray(vx1,windowsize,offset)
vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

yInput1,yLabel1=shapeArray(y1,windowsize,offset)
yInput2,yLabel2=shapeArray(y2,windowsize,offset)
vy1Input,vy1Label=shapeArray(vy1,windowsize,offset)
vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

xInput1_Test=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
yInput1_Test=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
vx1Input_Test=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
vy1Input_Test=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

xLabel1_Test=xLabel1[:,-1].reshape(xLabel1.shape[0],1)
xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
yLabel1_Test=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

zTest_in = np.concatenate([xInput1_Test, yInput1_Test, xInput2_Test, yInput2_Test, vx1Input_Test, vx2Input_Test, vy1Input_Test, vy2Input_Test], axis=2)

In [ ]:
positions = model.predict_on_batch(zTest_in)
print(positions.shape)

In [ ]:
x1_pos=positions[:,0]
y1_pos=positions[:,1]
x2_pos=positions[:,2]
y2_pos=positions[:,3]
print(x1_pos.shape[0])
print(x1[windowsize+offset-1:].shape)

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))
ax.plot(x1_pos,y1_pos,label=r"$m_1$ prediction", linestyle = '--')
ax.plot(x2_pos,y2_pos,label=r"$m_2$ prediction", linestyle = '--')
ax.plot(x1[windowsize+offset-1:], y1[windowsize+offset-1:],label=r"$m_1$ actual")
ax.plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual")
ax.plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", label="Initial position", c='k')

# Define the radius of the circle
r = L1 + L2

# Create points for the circle
theta = np.linspace(0, 2*np.pi, 100)  # Create 100 points between 0 and 2*pi
x_circle = r * np.cos(theta)  # Calculate x coordinates
y_circle = r * np.sin(theta)  # Calculate y coordinates

# Add the circle to the plot
plt.plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')  # Plot the circle

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Motion Prediction by Network")
ax.legend()

# Plot residual by finding the percentage difference between predicted and actual motions
fig,ax=plt.subplots()
actual_pos1 = np.sqrt(x1**2 + y1**2)
actual_pos2 = np.sqrt(x2**2 + y2**2)

x1_dif = x1_pos - x1[windowsize+offset-1:]
y1_dif = y1_pos - y1[windowsize+offset-1:]
pos1_dif = np.sqrt(x1_dif**2 + y1_dif**2) / actual_pos1[windowsize+offset-1:] * 100

x2_dif = x2_pos - x2[windowsize+offset-1:]
y2_dif = y2_pos - y2[windowsize+offset-1:]
pos2_dif = np.sqrt(x2_dif**2 + y2_dif**2) / actual_pos2[windowsize+offset-1:] * 100



ax.plot(pos1_dif,label=r"$m_1$")
ax.plot(pos2_dif,label=r"$m_2$")
ax.set_xlabel("Timestep")
ax.set_ylabel(r" $\Delta$% Position")
ax.set_title("Residual")
ax.legend()

That looks pretty accurate up until the 100th timestep (windowsize and offset cuts off first 40 timesteps) where our model is untrained. Lets try change some of the initial conditions. For $z_0$ we'll vary each position component by $\pm 10$ % and $\pm30$ % or add $\pm 10$ % and $ \pm30$ % of the position component onto the velocity, which will give us 16 new motion graphs.   

In [ ]:
# Create initial plot
fig,ax=plt.subplots(4, 4, figsize=(32,24))

# Create arr for change
variation_arr = np.array([1, 3, -1, -3]) * np.pi / 40

for i in range(4):
  for j, change in enumerate(variation_arr):
    z0[i] += change
    # Now we wil simulate the new projection with the varied z0 before testing it
    ret = solve_ivp(rhs, (0, tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
    z0[i] -= change
    z=ret.y
    theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]
    x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

    # Create shapeArrays
    xInput1,xLabel1=shapeArray(x1,windowsize,offset)
    xInput2,xLabel2=shapeArray(x2,windowsize,offset)
    vx1Input,vx1Label=shapeArray(vx1,windowsize,offset)
    vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

    yInput1,yLabel1=shapeArray(y1,windowsize,offset)
    yInput2,yLabel2=shapeArray(y2,windowsize,offset)
    vy1Input,vy1Label=shapeArray(vy1,windowsize,offset)
    vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

    xInput1_Test=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
    xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
    yInput1_Test=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
    yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
    vx1Input_Test=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
    vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
    vy1Input_Test=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
    vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

    xLabel1_Test=xLabel1[:,-1].reshape(xLabel1.shape[0],1)
    xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
    yLabel1_Test=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
    yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

    # Create testing input with new z0
    zTest_in = np.concatenate([xInput1_Test, yInput1_Test, xInput2_Test, yInput2_Test, vx1Input_Test, vx2Input_Test, vy1Input_Test, vy2Input_Test], axis=2)
    positions = model.predict_on_batch(zTest_in)

    x1_pos=positions[:,0]
    y1_pos=positions[:,1]
    x2_pos=positions[:,2]
    y2_pos=positions[:,3]

    ax[i, j].plot(x1_pos,y1_pos,label=r"$m_1$ prediction")
    ax[i, j].plot(x2_pos,y2_pos,label=r"$m_2$ prediction")
    ax[i, j].plot(x1[windowsize+offset-1:], y1[windowsize+offset-1:],label=r"$m_1$ actual", linestyle = '--')
    ax[i, j].plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual", linestyle = '--')
    ax[i, j].plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", c='k')
    ax[i, j].plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
    ax[i, j].set_xlabel("x")
    ax[i, j].set_ylabel("y")
    ax[i, j].set_xlim(-2.5,2.5)
    ax[i, j].set_ylim(-2.5,2.5)
    if change > 0:
      ax[i, j].set_title(f" $z_0[{i}]$ + {int(change * 40 / np.pi)}0%")
    else:
      ax[i, j].set_title(f" $z_0[{i}]$ - {int(abs((change * 40 / np.pi)))}0%")
    ax[i, j].legend()


The results look reasonable, with greater changes in initial conditions causing more chaos. We can see clearly that the model performs quite poorly for $m_1$ when we have any significant changes. It's harder to tell how accurate the model is for $m_2$ but it doesn't look too wildly off.

Let's now check how far into the future the model predict into. We initially predicting 20 timesteps ahead but now we will go upto 100.  

In [ ]:
# Create Timestep Array
timestep_arr = [20, 30, 50, 75,100]

fig,ax=plt.subplots(len(timestep_arr), 2, figsize=(12,24))

for index, timestep in enumerate(timestep_arr):

  model = keras.models.Sequential()
  model.add(keras.layers.Input(shape=(None, 8)))  # 8 input features per timestep for 2 masses
  model.add(keras.layers.LSTM(50, return_sequences=True))  # Increased units and return sequences
  model.add(keras.layers.Dropout(0.2)) # Added dropout for regularization
  model.add(keras.layers.LSTM(50, return_sequences=False))  # Use only the last output
  model.add(keras.layers.Dense(4, activation="linear"))  # 4 outputs
  model.compile(loss='mean_squared_error', optimizer='adam') # you can try using different optimizers or learning rates

  windowsize=timestep #Number of samples we will use to train our network
  offset=timestep #How many samples into the future to predict

  # Make cut off bigger so that we can accomadate bigger timesteps. Remember all models will be trained
  cutOff = 300  # with the same cutoff

  #Get our xInput and xLabel arrays
  xInput1,xLabel1=shapeArray(x1[:cutOff],windowsize,offset)
  xInput2,xLabel2=shapeArray(x2[:cutOff],windowsize,offset)
  vx1Input,vx1Label=shapeArray(vx1[:cutOff],windowsize,offset)
  vx2Input,vx2Label=shapeArray(vx2[:cutOff],windowsize,offset)


  #Get our yInput and yLabel arrays
  # We already have our outputs so we don't need to explicitly calculate them
  yInput1,yLabel1=shapeArray(y1[:cutOff],windowsize,offset)
  yInput2,yLabel2=shapeArray(y2[:cutOff],windowsize,offset)
  vy1Input,vy1Label=shapeArray(vy1[:cutOff],windowsize,offset)
  vy2Input,vy2Label=shapeArray(vy2[:cutOff],windowsize,offset)

  # Make training data
  xInput1_Train=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
  xInput2_Train=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
  yInput1_Train=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
  yInput2_Train=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
  vx1Input_Train=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
  vx2Input_Train=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
  vy1Input_Train=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
  vy2Input_Train=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

  xLabel1_Train=xLabel1[:,-1].reshape(xLabel1.shape[0],1) # added reshape to (batch_size, 1, 1)
  xLabel2_Train=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
  yLabel1_Train=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
  yLabel2_Train=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

  # Deepest array contains values from all 8 inputs
  y_in = np.concatenate([xInput1_Train, yInput1_Train, xInput2_Train, yInput2_Train, vx1Input_Train, vx2Input_Train, vy1Input_Train, vy2Input_Train], axis=2)
  y_target = np.concatenate([xLabel1_Train, yLabel1_Train, xLabel2_Train, yLabel2_Train], axis=1)

  steps=200  #Number of training steps
  costs=np.zeros(steps)  #Array for plotting cost
  for i in tqdm(range(steps)): # tqdm is progress bar
      costs[i]=model.train_on_batch(y_in,y_target) #Train the network

  # Create test set, we only create about labels but we make the inputs anyway
  xInput1,xLabel1=shapeArray(x1,windowsize,offset)
  xInput2,xLabel2=shapeArray(x2,windowsize,offset)
  vx1Input,vx1Label=shapeArray(vx1,windowsize,offset)
  vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

  yInput1,yLabel1=shapeArray(y1,windowsize,offset)
  yInput2,yLabel2=shapeArray(y2,windowsize,offset)
  vy1Input,vy1Label=shapeArray(vy1,windowsize,offset)
  vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

  xInput1_Test=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
  xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
  yInput1_Test=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
  yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
  vx1Input_Test=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
  vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
  vy1Input_Test=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
  vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

  xLabel1_Test=xLabel1[:,-1].reshape(xLabel1.shape[0],1)
  xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
  yLabel1_Test=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
  yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

  zTest_in = np.concatenate([xInput1_Test, yInput1_Test, xInput2_Test, yInput2_Test, vx1Input_Test, vx2Input_Test, vy1Input_Test, vy2Input_Test], axis=2)
  positions = model.predict_on_batch(zTest_in)

  x1_pos=positions[:,0]
  y1_pos=positions[:,1]
  x2_pos=positions[:,2]
  y2_pos=positions[:,3]

  ax[index, 0].plot(x1_pos,y1_pos,label=r"$m_1$ prediction")
  ax[index, 0].plot(x2_pos,y2_pos,label=r"$m_2$ prediction")
  ax[index, 0].plot(x1[windowsize+offset-1:], y1[windowsize+offset-1:],label=r"$m_1$ actual", linestyle = '--')
  ax[index, 0].plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual", linestyle = '--')
  ax[index, 0].plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", c='k')
  ax[index, 0].plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
  ax[index, 0].set_xlabel("x")
  ax[index, 0].set_ylabel("y")
  ax[index, 0].set_title(f" Predicted motion at $t=t_0 + {timestep} \delta t$")
  ax[index, 0].legend()

  # Plot Residual by finding the absolute differences in position
  actual_pos1 = np.sqrt(x1**2 + y1**2)
  actual_pos2 = np.sqrt(x2**2 + y2**2)

  x1_dif = x1_pos - x1[windowsize+offset-1:]
  y1_dif = y1_pos - y1[windowsize+offset-1:]
  pos1_dif = np.sqrt(x1_dif**2 + y1_dif**2) / actual_pos1[windowsize+offset-1:] * 100

  x2_dif = x2_pos - x2[windowsize+offset-1:]
  y2_dif = y2_pos - y2[windowsize+offset-1:]
  pos2_dif = np.sqrt(x2_dif**2 + y2_dif**2) / actual_pos2[windowsize+offset-1:] * 100

  ax[index,1].plot(pos1_dif,label=r"$m_1$")
  ax[index,1].plot(pos2_dif,label=r"$m_2$")
  ax[index,1].set_xlabel("Timestep")
  ax[index,1].set_ylabel(r" $\Delta$% Position")
  ax[index,1].set_title(f"Residual ($t=t_0 + {timestep} \delta t$)")
  ax[index,1].legend()



Looks pretty much the same, errors are quite small as well.

## Step 4

Let's now do everything we did before but with initial conditions $z_0$ = $[\pi/2, 0, \pi/2, 0]$, which should give a much more complex path.

In [ ]:
# This motion is going to me much more chaotic so we're going to use a more complex model
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(None, 8)))  # 8 input features per timestep for 2 masses
model.add(keras.layers.LSTM(75, return_sequences=True))  # Increased units and return sequences
model.add(keras.layers.Dropout(0.2)) # Added dropout for regularization
model.add(keras.layers.LSTM(75, return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(75, return_sequences=False))  # Use only the last output
model.add(keras.layers.Dense(4, activation="linear"))  # 4 outputs
model.compile(loss='mean_squared_error', optimizer='adam') # you can try using different optimizers or learning rates

In [ ]:
# Copy everything from above but change the initial conditions as stated in question 2
z0=[np.pi/2,0,np.pi/2,0]

#Time ranges
tmax, dt = 50, 0.1
t = np.arange(0, tmax+dt, dt)

# Solve initial value problem
ret = solve_ivp(rhs, (0,tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
z=ret.y

# Extract result
theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]
x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

windowsize=20 #Number of samples we will use to train our network
offset=20 #How many samples into the future to predict

# Define a cut off for training, rest will be used for prediction later on
cutOff = 200

#Get our xInput and xLabel arrays
xInput1,xLabel1=shapeArray(x1[:cutOff],windowsize,offset)
xInput2,xLabel2=shapeArray(x2[:cutOff],windowsize,offset)
vx1Input,vx1Label=shapeArray(vx1[:cutOff],windowsize,offset)
vx2Input,vx2Label=shapeArray(vx2[:cutOff],windowsize,offset)


#Get our yInput and yLabel arrays
# We already have our outputs so we don't need to explicitly calculate them
yInput1,yLabel1=shapeArray(y1[:cutOff],windowsize,offset)
yInput2,yLabel2=shapeArray(y2[:cutOff],windowsize,offset)
vy1Input,vy1Label=shapeArray(vy1[:cutOff],windowsize,offset)
vy2Input,vy2Label=shapeArray(vy2[:cutOff],windowsize,offset)

In [ ]:
# Reshape Arrays so for training so that they can fit into our network
xInput1_Train=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
xInput2_Train=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
yInput1_Train=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
yInput2_Train=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
vx1Input_Train=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
vx2Input_Train=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
vy1Input_Train=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
vy2Input_Train=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

xLabel1_Train=xLabel1[:,-1].reshape(xLabel1.shape[0],1) # added reshape to (batch_size, 1, 1)
xLabel2_Train=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
yLabel1_Train=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
yLabel2_Train=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

# Deepest array contains values from all 8 inputs
y_in = np.concatenate([xInput1_Train, yInput1_Train, xInput2_Train, yInput2_Train, vx1Input_Train, vx2Input_Train, vy1Input_Train, vy2Input_Train], axis=2)
y_target = np.concatenate([xLabel1_Train, yLabel1_Train, xLabel2_Train, yLabel2_Train], axis=1)

In [ ]:
steps=200  #Number of training steps
costs=np.zeros(steps)  #Array for plotting cost
for i in tqdm(range(steps)): # tqdm is progress bar
    costs[i]=model.train_on_batch(y_in,y_target) #Train the network

In [ ]:
# Create test set, we only create about labels but we make the inputs anyway
xInput1,xLabel1=shapeArray(x1,windowsize,offset)
xInput2,xLabel2=shapeArray(x2,windowsize,offset)
vx1Input,vx1Label=shapeArray(vx1,windowsize,offset)
vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

yInput1,yLabel1=shapeArray(y1,windowsize,offset)
yInput2,yLabel2=shapeArray(y2,windowsize,offset)
vy1Input,vy1Label=shapeArray(vy1,windowsize,offset)
vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

xInput1_Test=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
yInput1_Test=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
vx1Input_Test=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
vy1Input_Test=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

xLabel1_Test=xLabel1[:,-1].reshape(xLabel1.shape[0],1)
xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
yLabel1_Test=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

zTest_in = np.concatenate([xInput1_Test, yInput1_Test, xInput2_Test, yInput2_Test, vx1Input_Test, vx2Input_Test, vy1Input_Test, vy2Input_Test], axis=2)

In [ ]:
positions = model.predict_on_batch(zTest_in)
x1_pos=positions[:,0]
y1_pos=positions[:,1]
x2_pos=positions[:,2]
y2_pos=positions[:,3]

In [ ]:
fig,ax=plt.subplots()
ax.plot(x1_pos,y1_pos,label=r"$m_1$ prediction", linestyle = '--')
ax.plot(x2_pos,y2_pos,label=r"$m_2$ prediction", linestyle = '--')
ax.plot(x1[windowsize+offset-1:], y1[windowsize+offset-1:],label=r"$m_1$ actual")
ax.plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual")
ax.plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
ax.plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", label="Initial position", c='k')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Motion Prediction by Network")
ax.legend()

# Plot Residual by finding the absolute differences in position (not percentages this time)
fig,ax=plt.subplots()

x1_dif = x1_pos - x1[windowsize+offset-1:]
y1_dif = y1_pos - y1[windowsize+offset-1:]
pos1_dif = np.sqrt(x1_dif**2 + y1_dif**2)

x2_dif = x2_pos - x2[windowsize+offset-1:]
y2_dif = y2_pos - y2[windowsize+offset-1:]
pos2_dif = np.sqrt(x2_dif**2 + y2_dif**2)

ax.plot(pos1_dif,label=r"$m_1$")
ax.plot(pos2_dif,label=r"$m_2$")
ax.set_xlabel("Timestep")
ax.set_ylabel(r" $\Delta$ Position")
ax.set_title("Residual")
ax.legend()

In [ ]:
# Create initial plot
fig,ax=plt.subplots(4, 4, figsize=(32,24))

# Create arr for change
variation_arr = np.array([1, 3, -1, -3]) * np.pi / 40

for i in range(4):
  for j, change in enumerate(variation_arr):
    z0[i] += change
    # Now we wil simulate the new projection with the varied z0 before testing it
    ret = solve_ivp(rhs, (0, tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
    z0[i] -= change
    z=ret.y
    theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]
    x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

    # Create shapeArrays
    xInput1,xLabel1=shapeArray(x1,windowsize,offset)
    xInput2,xLabel2=shapeArray(x2,windowsize,offset)
    vx1Input,vx1Label=shapeArray(vx1,windowsize,offset)
    vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

    yInput1,yLabel1=shapeArray(y1,windowsize,offset)
    yInput2,yLabel2=shapeArray(y2,windowsize,offset)
    vy1Input,vy1Label=shapeArray(vy1,windowsize,offset)
    vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

    xInput1_Test=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
    xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
    yInput1_Test=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
    yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
    vx1Input_Test=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
    vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
    vy1Input_Test=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
    vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

    xLabel1_Test=xLabel1[:,-1].reshape(xLabel1.shape[0],1)
    xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
    yLabel1_Test=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
    yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

    # Create testing input with new z0
    zTest_in = np.concatenate([xInput1_Test, yInput1_Test, xInput2_Test, yInput2_Test, vx1Input_Test, vx2Input_Test, vy1Input_Test, vy2Input_Test], axis=2)
    positions = model.predict_on_batch(zTest_in)

    x1_pos=positions[:,0]
    y1_pos=positions[:,1]
    x2_pos=positions[:,2]
    y2_pos=positions[:,3]

    ax[i, j].plot(x1_pos,y1_pos,label=r"$m_1$ prediction")
    ax[i, j].plot(x2_pos,y2_pos,label=r"$m_2$ prediction")
    ax[i, j].plot(x1[windowsize+offset-1:], y1[windowsize+offset-1:],label=r"$m_1$ actual", linestyle = '--')
    ax[i, j].plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual", linestyle = '--')
    ax[i, j].plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
    ax[i, j].plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", c='k')
    ax[i, j].set_xlabel("x")
    ax[i, j].set_ylabel("y")
    ax[i, j].set_xlim(-2.5,2.5)
    ax[i, j].set_ylim(-2.5,2.5)
    if change > 0:
      ax[i, j].set_title(f" $z_0[{i}]$ + {int(change * 40 / np.pi)}0%")
    else:
      ax[i, j].set_title(f" $z_0[{i}]$ - {int(abs((change * 40 / np.pi)))}0%")
    ax[i, j].legend()

In [ ]:
# Create Timestep Array
timestep_arr = [20, 30, 50, 75,100]

fig,ax=plt.subplots(len(timestep_arr), 2, figsize=(16,25))

for index, timestep in enumerate(timestep_arr):

  model = keras.models.Sequential()
  model.add(keras.layers.Input(shape=(None, 8)))  # 8 input features per timestep for 2 masses
  model.add(keras.layers.LSTM(50, return_sequences=True))  # Increased units and return sequences
  model.add(keras.layers.Dropout(0.2)) # Added dropout for regularization
  model.add(keras.layers.LSTM(50, return_sequences=False))  # Use only the last output
  model.add(keras.layers.Dense(4, activation="linear"))  # 4 outputs
  model.compile(loss='mean_squared_error', optimizer='adam') # you can try using different optimizers or learning rates

  windowsize=timestep #Number of samples we will use to train our network
  offset=timestep #How many samples into the future to predict

  # Make cut off bigger so that we can accomadate bigger timesteps. Remember all models will be trained
  cutOff = 300  # with the same cutoff

  #Get our xInput and xLabel arrays
  xInput1,xLabel1=shapeArray(x1[:cutOff],windowsize,offset)
  xInput2,xLabel2=shapeArray(x2[:cutOff],windowsize,offset)
  vx1Input,vx1Label=shapeArray(vx1[:cutOff],windowsize,offset)
  vx2Input,vx2Label=shapeArray(vx2[:cutOff],windowsize,offset)


  #Get our yInput and yLabel arrays
  # We already have our outputs so we don't need to explicitly calculate them
  yInput1,yLabel1=shapeArray(y1[:cutOff],windowsize,offset)
  yInput2,yLabel2=shapeArray(y2[:cutOff],windowsize,offset)
  vy1Input,vy1Label=shapeArray(vy1[:cutOff],windowsize,offset)
  vy2Input,vy2Label=shapeArray(vy2[:cutOff],windowsize,offset)

  # Make training data
  xInput1_Train=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
  xInput2_Train=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
  yInput1_Train=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
  yInput2_Train=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
  vx1Input_Train=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
  vx2Input_Train=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
  vy1Input_Train=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
  vy2Input_Train=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

  xLabel1_Train=xLabel1[:,-1].reshape(xLabel1.shape[0],1) # added reshape to (batch_size, 1, 1)
  xLabel2_Train=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
  yLabel1_Train=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
  yLabel2_Train=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

  # Deepest array contains values from all 8 inputs
  y_in = np.concatenate([xInput1_Train, yInput1_Train, xInput2_Train, yInput2_Train, vx1Input_Train, vx2Input_Train, vy1Input_Train, vy2Input_Train], axis=2)
  y_target = np.concatenate([xLabel1_Train, yLabel1_Train, xLabel2_Train, yLabel2_Train], axis=1)

  steps=200  #Number of training steps
  costs=np.zeros(steps)  #Array for plotting cost
  for i in tqdm(range(steps)): # tqdm is progress bar
      costs[i]=model.train_on_batch(y_in,y_target) #Train the network

  # Create test set, we only create about labels but we make the inputs anyway
  xInput1,xLabel1=shapeArray(x1,windowsize,offset)
  xInput2,xLabel2=shapeArray(x2,windowsize,offset)
  vx1Input,vx1Label=shapeArray(vx1,windowsize,offset)
  vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

  yInput1,yLabel1=shapeArray(y1,windowsize,offset)
  yInput2,yLabel2=shapeArray(y2,windowsize,offset)
  vy1Input,vy1Label=shapeArray(vy1,windowsize,offset)
  vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

  xInput1_Test=xInput1.reshape(xInput1.shape[0],xInput1.shape[1],1)
  xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
  yInput1_Test=yInput1.reshape(yInput1.shape[0],yInput1.shape[1],1)
  yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
  vx1Input_Test=vx1Input.reshape(vx1Input.shape[0],vx1Input.shape[1],1)
  vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
  vy1Input_Test=vy1Input.reshape(vy1Input.shape[0],vy1Input.shape[1],1)
  vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

  xLabel1_Test=xLabel1[:,-1].reshape(xLabel1.shape[0],1)
  xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
  yLabel1_Test=yLabel1[:,-1].reshape(yLabel1.shape[0],1)
  yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

  zTest_in = np.concatenate([xInput1_Test, yInput1_Test, xInput2_Test, yInput2_Test, vx1Input_Test, vx2Input_Test, vy1Input_Test, vy2Input_Test], axis=2)
  positions = model.predict_on_batch(zTest_in)

  x1_pos=positions[:,0]
  y1_pos=positions[:,1]
  x2_pos=positions[:,2]
  y2_pos=positions[:,3]

  ax[index, 0].plot(x1_pos,y1_pos,label=r"$m_1$ prediction")
  ax[index, 0].plot(x2_pos,y2_pos,label=r"$m_2$ prediction")
  ax[index, 0].plot(x1[windowsize+offset-1:], y1[windowsize+offset-1:],label=r"$m_1$ actual", linestyle = '--')
  ax[index, 0].plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual", linestyle = '--')
  ax[index, 0].plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
  ax[index, 0].plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", c='k')
  ax[index, 0].set_xlabel("x")
  ax[index, 0].set_ylabel("y")
  ax[index, 0].set_title(f" Predicted motion at $t=t_0 + {timestep} \delta t$")
  ax[index, 0].legend()

  # Plot Residual by finding the absolute differences in position

  x1_dif = x1_pos - x1[windowsize+offset-1:]
  y1_dif = y1_pos - y1[windowsize+offset-1:]
  pos1_dif = np.sqrt(x1_dif**2 + y1_dif**2)

  x2_dif = x2_pos - x2[windowsize+offset-1:]
  y2_dif = y2_pos - y2[windowsize+offset-1:]
  pos2_dif = np.sqrt(x2_dif**2 + y2_dif**2)

  ax[index,1].plot(pos1_dif,label=r"$m_1$")
  ax[index,1].plot(pos2_dif,label=r"$m_2$")
  ax[index,1].set_xlabel("Timestep")
  ax[index,1].set_ylabel(r" $\Delta$ Position")
  ax[index,1].set_title(f"Residual ($t=t_0 + {timestep} \delta t$)")
  ax[index,1].legend()

## Step 5

The above projections look quite good but now we will see what happens when we only feed the mass of $m_2$ into our neural network. We should expect a more chaotic path from the network as we no longer have $m_1$ acting as an 'anchor' for our model.  

In [ ]:
# Use same models as before since we are trying to compare them I assume but only for 1 mass so change input and output
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(None, 4)))  # 4 input features per timestep for 1 mass
model.add(keras.layers.LSTM(50, return_sequences=True))  # Increased units and return sequences
model.add(keras.layers.Dropout(0.2)) # Added dropout for regularization
model.add(keras.layers.LSTM(50, return_sequences=False))  # Use only the last output
model.add(keras.layers.Dense(2, activation="linear"))  # 2 outputs
model.compile(loss='mean_squared_error', optimizer='adam') # you can try using different optimizers or learning rates

In [ ]:
windowsize=20 #Number of samples we will use to train our network
offset=20 #How many samples into the future to predict

# Define a cut off for training, rest will be used for prediction later on
cutOff = 200

# Same as before but only using m2
#Get our xInput and xLabel arrays
xInput2,xLabel2=shapeArray(x2[:cutOff],windowsize,offset)
vx2Input,vx2Label=shapeArray(vx2[:cutOff],windowsize,offset)

#Get our yInput and yLabel arrays
# We already have our outputs so we don't need to explicitly calculate them
yInput2,yLabel2=shapeArray(y2[:cutOff],windowsize,offset)
vy2Input,vy2Label=shapeArray(vy2[:cutOff],windowsize,offset)

# Reshape Arrays so for training so that they can fit into our network
xInput2_Train=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
yInput2_Train=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
vx2Input_Train=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
vy2Input_Train=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

xLabel2_Train=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
yLabel2_Train=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

# Deepest array contains values from all 8 inputs
y_in = np.concatenate([xInput2_Train, yInput2_Train, vx2Input_Train, vy2Input_Train], axis=2)
y_target = np.concatenate([xLabel2_Train, yLabel2_Train], axis=1)

In [ ]:
steps=200  #Number of training steps
costs=np.zeros(steps)  #Array for plotting cost
for i in tqdm(range(steps)): # tqdm is progress bar
    costs[i]=model.train_on_batch(y_in,y_target) #Train the network

#Plot costs vs steps
fig,ax=plt.subplots(figsize=(6,4))
ax.plot(np.arange(steps),costs,label=r"Costs")
ax.set_xlabel("Steps")
ax.set_ylabel("Cost")
ax.set_title("Network Training Cost")
ax.legend()

In [ ]:
# Create test set, we only create about labels but we make the inputs anyway
xInput2,xLabel2=shapeArray(x2,windowsize,offset)
vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

yInput2,yLabel2=shapeArray(y2,windowsize,offset)
vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

zTest_in = np.concatenate([xInput2_Test, yInput2_Test, vx2Input_Test, vy2Input_Test], axis=2)
positions = model.predict_on_batch(zTest_in)
x2_pos=positions[:,0]
y2_pos=positions[:,1]

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))
ax.plot(x2_pos,y2_pos,label=r"$m_2$ prediction", linestyle = '--')
ax.plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual")
ax.plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
ax.plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", label="Initial position", c='k')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Motion Prediction by Network")
ax.legend()

# Plot residual by finding the absolute difference
fig,ax=plt.subplots()

x2_dif = x2_pos - x2[windowsize+offset-1:]
y2_dif = y2_pos - y2[windowsize+offset-1:]
pos2_dif = np.sqrt(x2_dif**2 + y2_dif**2)

ax.plot(pos2_dif,label=r"$m_2$")
ax.set_xlabel("Timestep")
ax.set_ylabel(r" $\Delta$ Position")
ax.set_title("Residual")
ax.legend()

In [ ]:
# Create initial plot
fig,ax=plt.subplots(4, 4, figsize=(32,24))

# Create arr for change
variation_arr = np.array([1, 3, -1, -3]) * np.pi / 40

for i in range(len(z0)):
  for j, change in enumerate(variation_arr):
    z0[i] += change
    # Now we wil simulate the new projection with the varied z0 before testing it
    ret = solve_ivp(rhs, (0, tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
    z0[i] -= change
    z=ret.y
    theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]
    x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

    # Create shapeArrays
    xInput2,xLabel2=shapeArray(x2,windowsize,offset)
    vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

    yInput2,yLabel2=shapeArray(y2,windowsize,offset)
    vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

    xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
    yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
    vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
    vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

    xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
    yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

    # Create testing input with new z0
    zTest_in = np.concatenate([xInput2_Test, yInput2_Test, vx2Input_Test, vy2Input_Test], axis=2)
    positions = model.predict_on_batch(zTest_in)
    x2_pos=positions[:,0]
    y2_pos=positions[:,1]

    ax[i, j].plot(x2_pos,y2_pos,label=r"$m_2$ prediction")
    ax[i, j].plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual", linestyle = '--')
    ax[i, j].plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
    ax[i, j].plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", c='k')
    ax[i, j].set_xlabel("x")
    ax[i, j].set_ylabel("y")
    ax[i, j].set_xlim(-2.5,2.5)
    ax[i, j].set_ylim(-2.5,2.5)
    if change > 0:
      ax[i, j].set_title(f" $z_0[{i}]$ + {int(change * 40 / np.pi)}0%")
    else:
      ax[i, j].set_title(f" $z_0[{i}]$ - {int(abs((change * 40 / np.pi)))}0%")
    ax[i, j].legend()

In [ ]:
# Create Timestep Array
timestep_arr = [20, 30, 50, 75, 100]

fig,ax=plt.subplots(len(timestep_arr), 2, figsize=(16,25))

for index, timestep in enumerate(timestep_arr):

  model = keras.models.Sequential()
  model.add(keras.layers.Input(shape=(None, 4))) # 4 inputs
  model.add(keras.layers.LSTM(50, return_sequences=True))  # Increased units and return sequences
  model.add(keras.layers.Dropout(0.2)) # Added dropout for regularization
  model.add(keras.layers.LSTM(50, return_sequences=False))  # Use only the last output
  model.add(keras.layers.Dense(2, activation="linear"))  # 2 outputs
  model.compile(loss='mean_squared_error', optimizer='adam') # you can try using different optimizers or learning rates

  windowsize=timestep #Number of samples we will use to train our network
  offset=timestep #How many samples into the future to predict

  # Make cut off bigger so that we can accomadate bigger timesteps. Remember all models will be trained
  cutOff = 300  # with the same cutoff

  #Get our xInput and xLabel arrays
  xInput2,xLabel2=shapeArray(x2[:cutOff],windowsize,offset)
  vx2Input,vx2Label=shapeArray(vx2[:cutOff],windowsize,offset)


  #Get our yInput and yLabel arrays
  # We already have our outputs so we don't need to explicitly calculate them
  yInput2,yLabel2=shapeArray(y2[:cutOff],windowsize,offset)
  vy2Input,vy2Label=shapeArray(vy2[:cutOff],windowsize,offset)

  # Make training data
  xInput2_Train=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
  yInput2_Train=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
  vx2Input_Train=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
  vy2Input_Train=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

  xLabel2_Train=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
  yLabel2_Train=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

  # Deepest array contains values from all 8 inputs
  y_in = np.concatenate([xInput2_Train, yInput2_Train, vx2Input_Train, vy2Input_Train], axis=2)
  y_target = np.concatenate([xLabel2_Train, yLabel2_Train], axis=1)

  steps=200  #Number of training steps
  costs=np.zeros(steps)  #Array for plotting cost
  for i in tqdm(range(steps)): # tqdm is progress bar
      costs[i]=model.train_on_batch(y_in,y_target) #Train the network

  # Create test set, we only create about labels but we make the inputs anyway
  xInput2,xLabel2=shapeArray(x2,windowsize,offset)
  vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

  yInput2,yLabel2=shapeArray(y2,windowsize,offset)
  vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

  xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
  yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
  vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
  vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

  xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
  yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

  zTest_in = np.concatenate([xInput2_Test, yInput2_Test, vx2Input_Test, vy2Input_Test], axis=2)
  positions = model.predict_on_batch(zTest_in)

  x2_pos=positions[:,0]
  y2_pos=positions[:,1]

  ax[index, 0].plot(x2_pos,y2_pos,label=r"$m_2$ prediction")
  ax[index, 0].plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual", linestyle = '--')
  ax[index, 0].plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
  ax[index, 0].plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", c='k')
  ax[index, 0].set_xlabel("x")
  ax[index, 0].set_ylabel("y")
  ax[index, 0].set_title(f" Predicted motion at $t=t_0 + {timestep} \delta t$")
  ax[index, 0].legend()

  # Plot Residual by finding the absolute differences in position
  actual_pos2 = np.sqrt(x2**2 + y2**2)

  x2_dif = x2_pos - x2[windowsize+offset-1:]
  y2_dif = y2_pos - y2[windowsize+offset-1:]
  pos2_dif = np.sqrt(x2_dif**2 + y2_dif**2)

  ax[index,1].plot(pos2_dif,label=r"$m_2$")
  ax[index,1].set_xlabel("Timestep")
  ax[index,1].set_ylabel(r" $\Delta$ Position")
  ax[index,1].set_title(f"Residual ($t=t_0 + {timestep} \delta t$)")
  ax[index,1].legend()

### Step 4 (still in step 5)

In [ ]:
# This motion is going to me much more chaotic so we're going to use a more complex model
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(None, 4)))  # 4 input features per timestep for 1 mass
model.add(keras.layers.LSTM(75, return_sequences=True))  # Increased units and return sequences
model.add(keras.layers.Dropout(0.2)) # Added dropout for regularization
model.add(keras.layers.LSTM(75, return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(75, return_sequences=False))  # Use only the last output
model.add(keras.layers.Dense(2, activation="linear"))  # 2 Outputs
model.compile(loss='mean_squared_error', optimizer='adam') # you can try using different optimizers or learning rates

In [ ]:
# Copy everything from above but change the initial conditions as stated in question 2
z0=[np.pi/2,0,np.pi/2,0]

#Time ranges
tmax, dt = 50, 0.1
t = np.arange(0, tmax+dt, dt)

# Solve initial value problem
ret = solve_ivp(rhs, (0,tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
z=ret.y

# Extract result
theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]
x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

windowsize=20 #Number of samples we will use to train our network
offset=20 #How many samples into the future to predict

# Define a cut off for training, rest will be used for prediction later on
cutOff = 200

#Get our xInput and xLabel arrays
xInput2,xLabel2=shapeArray(x2[:cutOff],windowsize,offset)
vx2Input,vx2Label=shapeArray(vx2[:cutOff],windowsize,offset)

#Get our yInput and yLabel arrays
# We already have our outputs so we don't need to explicitly calculate them
yInput2,yLabel2=shapeArray(y2[:cutOff],windowsize,offset)
vy2Input,vy2Label=shapeArray(vy2[:cutOff],windowsize,offset)

In [ ]:
# Reshape Arrays so for training so that they can fit into our network
xInput2_Train=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
yInput2_Train=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
vx2Input_Train=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
vy2Input_Train=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

xLabel2_Train=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
yLabel2_Train=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

# Deepest array contains values from all 4 inputs
y_in = np.concatenate([xInput2_Train, yInput2_Train, vx2Input_Train, vy2Input_Train], axis=2)
y_target = np.concatenate([xLabel2_Train, yLabel2_Train], axis=1)

In [ ]:
steps=200  #Number of training steps
costs=np.zeros(steps)  #Array for plotting cost
for i in tqdm(range(steps)): # tqdm is progress bar
    costs[i]=model.train_on_batch(y_in,y_target) #Train the network

In [ ]:
# Create test set, we only create about labels but we make the inputs anyway
xInput2,xLabel2=shapeArray(x2,windowsize,offset)
vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

yInput2,yLabel2=shapeArray(y2,windowsize,offset)
vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

zTest_in = np.concatenate([xInput2_Test, yInput2_Test, vx2Input_Test, vy2Input_Test], axis=2)

In [ ]:
positions = model.predict_on_batch(zTest_in)
x2_pos=positions[:,0]
y2_pos=positions[:,1]

In [ ]:
fig,ax=plt.subplots()
ax.plot(x2_pos,y2_pos,label=r"$m_2$ prediction", linestyle = '--')
ax.plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual")
ax.plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
ax.plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", label="Initial position", c='k')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Motion Prediction by Network")
ax.legend()

# Plot Residual by finding the absolute differences in position (not percentages this time)
fig,ax=plt.subplots()

x2_dif = x2_pos - x2[windowsize+offset-1:]
y2_dif = y2_pos - y2[windowsize+offset-1:]
pos2_dif = np.sqrt(x2_dif**2 + y2_dif**2)

ax.plot(pos2_dif,label=r"$m_2$")
ax.set_xlabel("Timestep")
ax.set_ylabel(r" $\Delta$ Position")
ax.set_title("Residual")
ax.legend()

In [ ]:
# Create initial plot
fig,ax=plt.subplots(4, 4, figsize=(32,24))

# Create arr for change
variation_arr = np.array([1, 3, -1, -3]) * np.pi / 40

for i in range(4):
  for j, change in enumerate(variation_arr):
    z0[i] += change
    # Now we wil simulate the new projection with the varied z0 before testing it
    ret = solve_ivp(rhs, (0, tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
    z0[i] -= change
    z=ret.y
    theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]
    x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

    # Create shapeArrays
    xInput2,xLabel2=shapeArray(x2,windowsize,offset)
    vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

    yInput2,yLabel2=shapeArray(y2,windowsize,offset)
    vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

    xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
    yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
    vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
    vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

    xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
    yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

    # Create testing input with new z0
    zTest_in = np.concatenate([xInput2_Test, yInput2_Test, vx2Input_Test, vy2Input_Test], axis=2)
    positions = model.predict_on_batch(zTest_in)

    x2_pos=positions[:,0]
    y2_pos=positions[:,1]

    ax[i, j].plot(x2_pos,y2_pos,label=r"$m_2$ prediction")
    ax[i, j].plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual", linestyle = '--')
    ax[i, j].plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
    ax[i, j].plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", c='k')
    ax[i, j].set_xlabel("x")
    ax[i, j].set_ylabel("y")
    ax[i, j].set_xlim(-2.5,2.5)
    ax[i, j].set_ylim(-2.5,2.5)
    if change > 0:
      ax[i, j].set_title(f" $z_0[{i}]$ + {int(change * 40 / np.pi)}0%")
    else:
      ax[i, j].set_title(f" $z_0[{i}]$ - {int(abs((change * 40 / np.pi)))}0%")
    ax[i, j].legend()

In [ ]:
# Create Timestep Array
timestep_arr = [20, 30, 50, 75,100]

fig,ax=plt.subplots(len(timestep_arr), 2, figsize=(16,25))

for index, timestep in enumerate(timestep_arr):

  model = keras.models.Sequential()
  model.add(keras.layers.Input(shape=(None, 4)))  # 4 inputs
  model.add(keras.layers.LSTM(50, return_sequences=True))  # Increased units and return sequences
  model.add(keras.layers.Dropout(0.2)) # Added dropout for regularization
  model.add(keras.layers.LSTM(50, return_sequences=False))  # Use only the last output
  model.add(keras.layers.Dense(2, activation="linear"))  # 2 outputs
  model.compile(loss='mean_squared_error', optimizer='adam') # you can try using different optimizers or learning rates

  windowsize=timestep #Number of samples we will use to train our network
  offset=timestep #How many samples into the future to predict

  # Make cut off bigger so that we can accomadate bigger timesteps. Remember all models will be trained
  cutOff = 300  # with the same cutoff

  #Get our xInput and xLabel arrays
  xInput2,xLabel2=shapeArray(x2[:cutOff],windowsize,offset)
  vx2Input,vx2Label=shapeArray(vx2[:cutOff],windowsize,offset)


  #Get our yInput and yLabel arrays
  # We already have our outputs so we don't need to explicitly calculate them
  yInput2,yLabel2=shapeArray(y2[:cutOff],windowsize,offset)
  vy2Input,vy2Label=shapeArray(vy2[:cutOff],windowsize,offset)

  # Make training data
  xInput2_Train=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
  yInput2_Train=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
  vx2Input_Train=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
  vy2Input_Train=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

  xLabel2_Train=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
  yLabel2_Train=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

  # Deepest array contains values from all 8 inputs
  y_in = np.concatenate([xInput2_Train, yInput2_Train, vx2Input_Train, vy2Input_Train], axis=2)
  y_target = np.concatenate([xLabel2_Train, yLabel2_Train], axis=1)

  steps=200  #Number of training steps
  costs=np.zeros(steps)  #Array for plotting cost
  for i in tqdm(range(steps)): # tqdm is progress bar
      costs[i]=model.train_on_batch(y_in,y_target) #Train the network

  # Create test set, we only create about labels but we make the inputs anyway
  xInput2,xLabel2=shapeArray(x2,windowsize,offset)
  vx2Input,vx2Label=shapeArray(vx2,windowsize,offset)

  yInput2,yLabel2=shapeArray(y2,windowsize,offset)
  vy2Input,vy2Label=shapeArray(vy2,windowsize,offset)

  xInput2_Test=xInput2.reshape(xInput2.shape[0],xInput2.shape[1],1)
  yInput2_Test=yInput2.reshape(yInput2.shape[0],yInput2.shape[1],1)
  vx2Input_Test=vx2Input.reshape(vx2Input.shape[0],vx2Input.shape[1],1)
  vy2Input_Test=vy2Input.reshape(vy2Input.shape[0],vy2Input.shape[1],1)

  xLabel2_Test=xLabel2[:,-1].reshape(xLabel2.shape[0],1)
  yLabel2_Test=yLabel2[:,-1].reshape(yLabel2.shape[0],1)

  zTest_in = np.concatenate([xInput2_Test, yInput2_Test, vx2Input_Test, vy2Input_Test], axis=2)
  positions = model.predict_on_batch(zTest_in)

  x2_pos=positions[:,0]
  y2_pos=positions[:,1]

  ax[index, 0].plot(x2_pos,y2_pos,label=r"$m_2$ prediction")
  ax[index, 0].plot(x2[windowsize+offset-1:], y2[windowsize+offset-1:],label=r"$m_2$ actual", linestyle = '--')
  ax[index, 0].plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", c='k')
  ax[index, 0].plot(x_circle, y_circle, linestyle='--', label=f'Circle (r= $L_1$ + $L_2$)')
  ax[index, 0].set_xlabel("x")
  ax[index, 0].set_ylabel("y")
  ax[index, 0].set_title(f" Predicted motion at $t=t_0 + {timestep} \delta t$")
  ax[index, 0].legend()

  # Plot Residual by finding the absolute differences in position

  x2_dif = x2_pos - x2[windowsize+offset-1:]
  y2_dif = y2_pos - y2[windowsize+offset-1:]
  pos2_dif = np.sqrt(x2_dif**2 + y2_dif**2)

  ax[index,1].plot(pos2_dif,label=r"$m_2$")
  ax[index,1].set_xlabel("Timestep")
  ax[index,1].set_ylabel(r" $\Delta$ Position")
  ax[index,1].set_title(f"Residual ($t=t_0 + {timestep} \delta t$)")
  ax[index,1].legend()